## Retail Dashboard

In [ ]:
# Visualization and dashboard were created using Redash (https://redash.io/)

**Create a table for our retail data**

In [ ]:
CREATE TABLE test.retail (
    InvoiceNo stringMergeTree,
    StockCode stringMergeTree,
    Description stringMergeTree,
    Quantity Int32,
    InvoiceDate DateTime('Europe/London'),
    UnitPrice Decimal64(3),
    CustomerID uint32,
    Country stringMergeTree)
ENGINE = MergeTree
ORDER BY InvoiceDate, CustomerID

**Check the number of unique customers by country**

In [ ]:
SELECT COUNT(DISTINCT CustomerID) AS uniq_customers,
       Country
FROM default.retail
GROUP BY Country

<img src='task1.png'>

**Check the change of MAU (monthly active users) in UK, Australia and Netherlands**

In [ ]:
SELECT COUNT(DISTINCT CustomerID) AS active_users,
       toStartOfMonth(InvoiceDate) AS MONTH,
       Country AS country
FROM default.retail
WHERE country IN ('United Kingdom',
                  'Australia',
                  'Netherlands')
GROUP BY MONTH,
         country
ORDER BY MONTH

<img src='task2.png'>

**Check the change of MAU (monthly active users) for all countries except UK**

In [ ]:
SELECT COUNT(DISTINCT CustomerID) AS active_users,
       toStartOfMonth(InvoiceDate) AS MONTH,
       Country AS country
FROM default.retail
WHERE country != 'United Kingdom'
GROUP BY MONTH,
         country
ORDER BY MONTH

<img src='task3.png'>

**Calculate AOV (average order value) for each country**

In [ ]:
SELECT AVG(order_value) AS AOV,
       Country
FROM
  (SELECT InvoiceNo,
          SUM(TotalPrice) AS order_value,
          Country
   FROM
     (SELECT InvoiceNo,
             Quantity * UnitPrice AS TotalPrice,
             Country
      FROM default.retail)
   GROUP BY InvoiceNo,
            Country
   ORDER BY order_value DESC)
GROUP BY Country
ORDER BY AOV DESC

<img src='task_4.png'>

**Check the dynamics of AOV metrics for United Kingdom, Germany, France, Spain, Netherlands, Belgium, Switzerland, Portugal, Australia, USA**

In [ ]:
SELECT AVG(order_value) AS AOV,
       Country,
       MONTH
FROM
  (SELECT SUM(TotalPrice) AS order_value,
          MONTH,
          Country
   FROM
     (SELECT InvoiceNo,
             Quantity * UnitPrice AS TotalPrice,
             Country,
             toStartOfMonth(InvoiceDate) AS MONTH
      FROM default.retail
      WHERE Country IN ('United Kingdom',
                        'Germany',
                        'France',
                        'Spain',
                        'Netherlands',
                        'Belgium',
                        'Switzerland',
                        'Portugal',
                        'Australia',
                        'USA') )
   GROUP BY InvoiceNo,
            Country,
            MONTH
   ORDER BY MONTH ASC, order_value DESC)
GROUP BY MONTH,
         Country
ORDER BY MONTH ASC, AOV DESC

<img src='task_5.png'>

**Calculate an average number of items per order by country**

In [ ]:
SELECT AVG(quantity_per_invoice) AS average_items,
       Country
FROM
  (SELECT InvoiceNo,
          SUM(Quantity) AS quantity_per_invoice,
          Country
   FROM default.retail
   GROUP BY InvoiceNo,
            Country)
GROUP BY Country
ORDER BY average_items DESC

<img src='task_6.png'>

**Investigate customers from the Netherlands. Find a customer who has bought the biggest number of items**

In [ ]:
SELECT
    SUM(Quantity) AS overall_quantity,
    CustomerID
FROM default.retail
WHERE Country == 'Netherlands'
GROUP BY CustomerID
ORDER BY overall_quantity DESC
LIMIT 100

Now we can see that an average number of items per invoice is so high for customers from the Netherlands because we have one customer (CustomerID 14646) who has purchased 196719 items!
<img src='task_7.png'>

**Calculate revenue deistribution per country**

In [ ]:
SELECT
    Country,
    --UnitPrice, 
    --Quantity,
    SUM(UnitPrice * Quantity) AS total_revenue
FROM default.retail
GROUP BY Country
ORDER BY total_revenue DESC

**Calculate monthly revenue dynamics for UK**

In [ ]:
SELECT
    toStartOfMonth(InvoiceDate) AS month,
    SUM(UnitPrice * Quantity) AS monthly_reveue
FROM default.retail
WHERE Country == 'United Kingdom'
    AND month < '2011-12-01'
GROUP BY month
ORDER BY month ASC
LIMIT 100

**Find the top-10 sales value items in UK in November**

In [ ]:
SELECT
    Description,
    SUM(Quantity) AS items_sold,
    UnitPrice,
    items_sold * UnitPrice AS total_sales_revenue
FROM default.retail
WHERE (toStartOfMonth(InvoiceDate) == '2011-11-01')
    AND Country == 'United Kingdom'
GROUP BY Description, UnitPrice
ORDER BY total_sales_revenue DESC
LIMIT 10

**Create a dashboard using the last three queries**

<img src='task_8_dashboard.png'>

#### Dashboard summary
The geography of the store's sales is shown in the upper left part of the dashboard. These are mainly Western Europe, Scandinavian countries, Great Britain, Canada, Australia and Brazil. The breakdown of revenue by country is shown in the bottom left corner. The first place by revenue is occupied by Great Britain. Moreover, the revenue indicators in Great Britain for the reporting period exceeded the revenue in the Netherlands (the second place) by 23.78 times.  

Since the major part of the store's revenue is generated in the UK, an analysis of sales in this country was carried out on the right side of the dashboard. The top right corner shows the breakdown of revenue by month. It can be seen that the peak of sales is in November. This is most likely due to the eve of the Christmas holidays. After analyzing the products sold in November, we can see that the top 10 items sold in terms of revenue include Christmas decorations, garlands and New Year's gifts (frames, thermo mugs, red Christmas bags). Details on cost, quantity, as well as total revenue from the top 10 products in November are shown on the dashboard.  

Therefore, it is recommended to consider:  
- avoiding a shortage in warehouses of Christmas goods in the month of November,
- to diversify the assortment of the store, to conduct campaigns to attract customers in order to avoid a significant seasonal decline in revenue in the the UK (the main market) in the period from January to September.  